# FPM Simulation
This ntoebook is not fully functional.

In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import llops as yp
import comptic
import llops.operators as ops
from tqdm.notebook import tqdm

from comptic import fpm

yp.setDefaultDatatype('complex32')
yp.setDefaultBackend('numpy')


# Generate Object

In [2]:
object_shape = (256, 256)
object_true = yp.simulation.ucb(object_shape)

plt.figure()
plt.subplot(121)
plt.imshow(yp.abs(object_true))
plt.title('Amplitude')
plt.subplot(122)
plt.imshow(yp.angle(object_true))
plt.title('Phase')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Phase')

## Define Settings

In [3]:
# Generate system parameters
system_parameters = fpm.getDefaultSystemParams(pixel_count=object_shape)

# Show led positions
fpm.plotLedPattern( **system_parameters)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Generate Pupil Function

In [4]:
# Generate Pupil
pupil = comptic.imaging.pupil(object_shape, **system_parameters)

## Generate LED Sequences

In [5]:
# Select LED pattern. Should be a list of list of integers; each internal list has the LED id positions which are illuminated.
single = [[i] for i, x in enumerate(system_parameters['illumination_source_position_list_na'])]
multiplexed = [[4 * i + j for j in range(4)] for i in range(8)]
patterns = single[:120]

## Generate Forward Models

In [6]:
# Generate Intensity Operator
I = ops.Intensity(object_shape)

# Generate Phase Ramp Operator
R = ops.PhaseRamp(object_shape)

# Generate Fourier Transform Operator
F = ops.FourierTransform(object_shape)

# Generate pupil dot multiplication operator
P = ops.Diagonalize(pupil, label='\\tilde P')

# Parse Illumination wavelength
illumination_wavelength = system_parameters['illumination_wavelength']

# Convert led position list to correct backend
led_position_list_na = yp.asarray(system_parameters['illumination_source_position_list_na'], dtype='complex32')

# Derived constants
effective_pixel_size = system_parameters['camera_pixel_size'] / system_parameters['objective_magnification'] 
dkxy = yp.asarray([1 / (sz * effective_pixel_size) for sz in object_shape], dtype='complex32')

# Initialise forward model list (A_list) and measurement list (y_list)
A_list, y_list = [], []
for pattern_index, led_pattern in enumerate(tqdm(patterns, 'Forward Models Generated')):
    A_pattern = None
    
    # Loop over all LEDs in this pattern
    for led_id in led_pattern:
        led_position_na = yp.squeeze(led_position_list_na[led_id])
        
        # Determine the shift of the LED position in pixels
        pixel_shift = led_position_na / illumination_wavelength / dkxy
        
        # Create phase ramp multiplication operator
        S = ops.Diagonalize(R * pixel_shift, label='S_{' + str(pattern_index) + '}')
        
        # Generate complete forward model
        _A = I * (F.H * P * F * S)
        
        # Add this forward operator to the list
        A_pattern = _A if not A_pattern else A_pattern + _A
    
    # Append forward model to list
    A_list.append(A_pattern)
    
    # Generate measurement
    y_list.append(A_pattern * object_true)
    
# Combine these operators into a set if vertically stacked measurements
A = ops.Vstack(A_list)

# Create measurements
y = ops.VecStack(y_list)

## Create Objective Function

In [9]:
objective = yp.solvers.objectivefunctions.L2(A, y) + yp.solvers.regularizers.WaveletSparsity(object_shape)

# Render Latex
objective.latex()

AttributeError: 'Operator' object has no attribute 'sum'

In [87]:
# Define Initialization
initialization = yp.ones(object_shape)

# Solve by accelerated gradient descent
x_opt = yp.solvers.Fista(objective).solve(iteration_count=20, display_type='text', step_size=5e-3,
                                          use_nesterov_acceleration=True, nesterov_restart_enabled=True, initialization=initialization)

# Show recovered object
plt.figure()
plt.subplot(121)
plt.imshow(np.abs(x_opt), cmap = 'gray')
plt.title('Reconstructed Amplitude')

plt.subplot(122)
plt.imshow(np.angle(x_opt), cmap = 'gray')
plt.title('Reconstructed Phase')

plt.tight_layout()
plt.show()

Minimizing function:


<IPython.core.display.Latex object>

|  Iter  |      Cost      | Elapsed time (s) |  Norm of Step  | Memory Usage (CPU/GPU) |
+ ------ + -------------- + ---------------- + -------------- + ---------------------- +
|    0   |    1.18e+05    |       0.00       |    0.00e+00    |  344.5 MB /  302.0 MB  |
|    2   |    7.69e+04    |       2.10       |    1.26e+00    |  345.4 MB /  408.8 MB  |
|    4   |    6.35e+04    |       4.51       |    1.26e+00    |  347.0 MB /  528.8 MB  |
|    6   |    5.78e+04    |       7.08       |    5.85e-01    |  345.6 MB /  306.8 MB  |
|    8   |    5.38e+04    |       9.53       |    1.11e-01    |  346.2 MB /  412.8 MB  |
|   10   |    5.10e+04    |      12.28       |    4.00e-01    |  347.1 MB /  508.8 MB  |
|   12   |    4.96e+04    |      15.05       |    2.74e-01    |  350.5 MB /  615.8 MB  |
|   14   |    4.89e+04    |      17.78       |    6.33e-02    |  349.5 MB /  306.8 MB  |
|   16   |    4.86e+04    |      20.20       |    2.30e-02    |  349.6 MB /  402.3 MB  |
|   18   |    4.85e+0

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …